In [1]:
!pip install fpdf2
!pip install qrcode[pil]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.4/206.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.3 MB/s eta 0:00:00


In [10]:
from fpdf import FPDF
from fpdf.enums import XPos, YPos

def generate_pdf():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('helvetica', size=12)
    pdf.cell(200, 10, text="Medical Prescription", new_x=XPos.RIGHT, new_y=YPos.TOP, align='C')

    # Example medical prescription content
    patient_name = "John Doe"
    medicines = ["Medicine A", "Medicine B", "Medicine C"]

    pdf.ln(20)
    pdf.cell(200, 10, text=f"Patient Name: {patient_name}", new_x=XPos.RIGHT, new_y=YPos.NEXT)
    pdf.ln(10)
    pdf.cell(200, 10, text="Prescribed Medicines:", new_x=XPos.RIGHT, new_y=YPos.NEXT)
    pdf.ln(10)
    for medicine in medicines:
        pdf.cell(200, 10, text=f"- {medicine}", new_x=XPos.RIGHT, new_y=YPos.NEXT)

    pdf_file_path = 'medical_prescription.pdf'
    pdf.output(pdf_file_path)
    return pdf_file_path

# Generate PDF
pdf_file_path = generate_pdf()
pdf_file_path


'medical_prescription.pdf'

In [3]:
import qrcode
from PIL import Image
from flask import Flask, send_file
import io
import datetime
import os
import threading

app = Flask(__name__)

# Generate PDF (using fpdf2) - This is similar to the previous example
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Medical Prescription', align='C', ln=1)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

    def add_prescription(self, patient_name, doctor_name, medicines):
        self.set_font('Arial', '', 12)
        self.cell(0, 10, f'Patient Name: {patient_name}', ln=1)
        self.cell(0, 10, f'Doctor: {doctor_name}', ln=1)
        self.ln(10)

        self.set_font('Arial', 'I', 12)
        self.cell(0, 10, 'Medicines:', ln=1)
        self.ln(5)

        self.set_font('Arial', '', 12)
        for medicine in medicines:
            self.cell(0, 10, f'- {medicine}', ln=1)

# Generate and serve the PDF file
def generate_pdf():
    pdf = PDF()
    pdf.add_page()
    patient_name = 'John Doe'
    doctor_name = 'Dr. Smith'
    medicines = [
        'Paracetamol 500mg - 1 tablet, three times a day after meals',
        'Amoxicillin 250mg - 1 capsule, twice a day for 7 days'
    ]
    pdf.add_prescription(patient_name, doctor_name, medicines)

    pdf_file_path = 'medical_prescription.pdf'
    pdf.output(pdf_file_path)
    return pdf_file_path

In [4]:
# Endpoint to serve the time-limited PDF file
@app.route('/download_pdf')
def download_pdf():
    pdf_file_path = generate_pdf()
    return send_file(pdf_file_path, as_attachment=True)

In [5]:
# Generate QR Code for the time-limited PDF download URL
def generate_qr_code():
    download_url = 'https://chat.openai.com/'
    qr = qrcode.QRCode(version=1, box_size=10, border=5)
    qr.add_data(download_url)
    qr.make(fit=True)

    qr_file_path = 'prescription_qr.png'
    qr_img = qr.make_image(fill='black', back_color='white')
    qr_img.save(qr_file_path)
    print(f'QR Code saved as "{qr_file_path}"')

In [6]:
# Function to remove the QR code file after expiration
def remove_qr_code():
    qr_file_path = 'prescription_qr.png'
    if os.path.exists(qr_file_path):
        os.remove(qr_file_path)
        print(f'Removed expired QR Code: "{qr_file_path}"')

# Schedule QR code removal after 10 minutes
def schedule_qr_code_removal():
    threading.Timer(300, remove_qr_code).start()  # 600 seconds = 10 minutes

# Main function to start the Flask app and generate QR code
if __name__ == '__main__':
    generate_qr_code()
    schedule_qr_code_removal()
    app.run()


QR Code saved as "prescription_qr.png"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
